# Hopital Patients Claims 
## Landing to Raw 

####  Run this cell to set up and start your interactive session.


In [ ]:
%idle_timeout 10
%glue_version 5.0
%worker_type G.1X
%number_of_workers 2

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql.functions import * 
from pyspark.sql.types import * 
from awsglue import DynamicFrame
import json 
from datetime import date
import boto3
from botocore.exceptions import ClientError
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [ ]:
todays_date = datetime.today().strftime('%Y/%m/%d/')
base_landing_path = 's3://hospital-patients-claims-bucket/'
files_location = base_landing_path+f"{todays_date}"

In [ ]:
df = glueContext.create_data_frame.from_options(connection_type='s3', 
                                                connection_options={"paths": [base_landing_path], "recurse": True},
                                                format='json') 

In [ ]:
df = df.withColumn('source_file_path',input_file_name())
df = df.withColumn('source_load_path',lit(files_location))
df = df.withColumn('load_date',lit(current_date()))
df = df.withColumn('load_timestamp',lit(current_timestamp()))

In [ ]:
s3output = glueContext.getSink(
    path='s3://hospital-patients-claims-bucket/raw_zone/raw_hospital_patients_claims',
    connection_type="s3",
    updateBehavior="UPDATE_IN_DATABASE",
    partitionKeys=['load_date'],
    enableUpdateCatalog=True
)
s3output.setCatalogInfo(
    catalogDatabase="hospital_patients_claims", catalogTableName="raw_hospital_patients_claims"
)
s3output.setFormat("glueparquet")
s3output.writeFrame(DynamicFrame.fromDF(df, glueContext))
